In [25]:
import numpy as np
import tensorflow as tf

import plotly.graph_objects as go

In [165]:
num_quasimomenta = 3

max_momentum_site = 11
num_momentum_sites = (2*max_momentum_site + 1) ** 2

In [166]:
quasimomentum = np.mgrid[0:2:num_quasimomenta*1j,
                         0:2:num_quasimomenta*1j].reshape(2,-1).T

momentum_site = np.mgrid[-max_momentum_site:max_momentum_site+1:1,
                         -max_momentum_site:max_momentum_site+1:1].reshape(2,-1).T

momentum = 2*momentum_site + quasimomentum[:, np.newaxis]
kinetic = np.linalg.norm(momentum, axis=-1)**2

momentum_site_change = momentum_site[:, np.newaxis] - momentum_site
max_momentum_site_change = 2 * max_momentum_site

In [197]:
# spatial_potential = np.array(
#     [[-1, -1, -1],
#      [-1,  0, -1],
#      [-1, -1, -1]]
# )

spatial_potential = np.array(
    [[-1, -1, -1, -1, -1, -1, -1, -1, -1],
     [-1,  0, -1,  0, -1,  0, -1,  0, -1],
     [-1, -1, -1, -1, -1, -1, -1, -1, -1],
     [-1,  0, -1,  0, -1,  0, -1,  0, -1],
     [-1, -1, -1, -1, -1, -1, -1, -1, -1],
     [-1,  0, -1,  0, -1,  0, -1,  0, -1],
     [-1, -1, -1, -1, -1, -1, -1, -1, -1],
     [-1,  0, -1,  0, -1,  0, -1,  0, -1],
     [-1, -1, -1, -1, -1, -1, -1, -1, -1]]
)

In [198]:
momentum_site_potential = np.fft.fft2(spatial_potential)

In [199]:
max_potential_momentum_site = (momentum_site_potential.shape[0] - 1) // 2

potential = np.pad(momentum_site_potential, max_momentum_site_change - max_potential_momentum_site)[
    momentum_site_change[:, :, 0] + max_momentum_site_change,
    momentum_site_change[:, :, 1] + max_momentum_site_change,
]

energy = tf.cast(tf.linalg.diag(kinetic), dtype=tf.complex128) + potential[np.newaxis, ...]

dispersion = tf.reshape(tf.math.real(tf.linalg.eigvalsh(energy)), (num_quasimomenta, num_quasimomenta, num_momentum_sites))


In [200]:
figure = go.Figure(data=[go.Surface(z=dispersion[:, :, band], showscale=False) for band in range(0, 3)])
figure.update_layout(autosize=False, width=750, height=750)
figure.show()